# Data Setni Yuklash 

In [8]:
import pandas as pd 
import numpy as np 
import seaborn as sns 

df = sns.load_dataset("titanic")

df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


# Missing values

In [10]:
df["age"] = df['age'].fillna(df["age"].median())
df["embarked"] = df["embarked"].fillna(df["embarked"].mode()[0])
df = df.drop(columns=["deck"])
df = df.dropna()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     889 non-null    int64   
 1   pclass       889 non-null    int64   
 2   sex          889 non-null    object  
 3   age          889 non-null    float64 
 4   sibsp        889 non-null    int64   
 5   parch        889 non-null    int64   
 6   fare         889 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        889 non-null    category
 9   who          889 non-null    object  
 10  adult_male   889 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        889 non-null    object  
 13  alone        889 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 86.1+ KB


In [12]:
df.to_csv(r"C:\Users\Rasulbek907\Desktop\Project_One\Data\Preprocessed_Data\Missing_Values.csv")

# Feature Enginering

In [13]:
df["family_size"] = df["sibsp"] + df["parch"] + 1
df["is_alone"] = np.where(df["family_size"] == 1, 1, 0)
df["age_group"] = pd.cut(df["age"], bins= [0, 12, 18, 35, 60, 100], labels=["child", "teen", "young", "adult", "senior"])
df["fare_per_person"] = df["fare"] / df["family_size"]
df["title"] = df["who"]
df = df.drop(columns=["alive", "class", "who"])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   survived         889 non-null    int64   
 1   pclass           889 non-null    int64   
 2   sex              889 non-null    object  
 3   age              889 non-null    float64 
 4   sibsp            889 non-null    int64   
 5   parch            889 non-null    int64   
 6   fare             889 non-null    float64 
 7   embarked         889 non-null    object  
 8   adult_male       889 non-null    bool    
 9   embark_town      889 non-null    object  
 10  alone            889 non-null    bool    
 11  family_size      889 non-null    int64   
 12  is_alone         889 non-null    int64   
 13  age_group        889 non-null    category
 14  fare_per_person  889 non-null    float64 
 15  title            889 non-null    object  
dtypes: bool(2), category(1), float64(3), int64(6), ob

In [15]:
df.to_csv(r"C:\Users\Rasulbek907\Desktop\Project_One\Data\Enginered_Data\Feature_Enginering.csv")

# Feature Selection

In [17]:
x = df.drop("survived", axis=1)
y = df["survived"]

In [18]:
import matplotlib.pyplot as plt 
corr = df.corr(numeric_only=True)
print(corr["survived"].sort_values(ascending=False))

survived           1.000000
fare               0.255290
fare_per_person    0.217902
parch              0.083151
family_size        0.018277
sibsp             -0.034040
age               -0.069822
is_alone          -0.206207
alone             -0.206207
pclass            -0.335549
adult_male        -0.555520
Name: survived, dtype: float64


In [19]:
selected_num = corr["survived"][abs(corr["survived"]) > 0.05 ].index.tolist()
selected_num.remove("survived")

In [20]:
cat_cols = x.select_dtypes(include="object").columns.tolist()
x = df[selected_num + cat_cols]

# encoding

In [21]:
x = pd.get_dummies(x, drop_first=True)

# scaling

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_scaled = scaler.fit_transform(x)

# train test Split qilish 

In [23]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42, stratify=y)

# bir nechta algaritmar bilan natijani sinaymiz 

In [24]:
from sklearn.linear_model  import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# LogisticRegression

In [25]:
log_model = LogisticRegression(max_iter=1000)
log_model.fit(x_train, y_train)

log_pred = log_model.predict(x_test)
print("LogisticRegression", accuracy_score(y_test, log_pred))

LogisticRegression 0.8089887640449438


# DecisionTreeClassifier

In [27]:
dec_model = DecisionTreeClassifier()
dec_model.fit(x_train, y_train)

dec_pred = dec_model.predict(x_test)
print("DecisionTreeClassifier", accuracy_score(y_test, dec_pred))

DecisionTreeClassifier 0.7752808988764045


# KNeighborsClassifier

In [28]:
knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)

knn_pred = knn_model.predict(x_test)
print("KNeighborsClassifier", accuracy_score(y_test, knn_pred))

KNeighborsClassifier 0.7921348314606742


# SVC

In [29]:
svc_model = SVC()
svc_model.fit(x_train, y_train)

svc_pred = svc_model.predict(x_test)
print("SVC", accuracy_score(y_test, svc_pred))

SVC 0.8202247191011236


# Hyperparameter tuning

In [30]:
import optuna 
from sklearn.model_selection import cross_val_score

c:\Users\Rasulbek907\Desktop\Project_One\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
def objective(trial):

    C = trial.suggest_float("C", 0.01, 10)
    solver = trial.suggest_categorical("solver", ["lbfgs", "liblinear"])

    

    model = LogisticRegression( C=C, solver=solver, max_iter=1000)

    score = cross_val_score(
        model, 
        x_scaled, 
        y, 
        cv=5,
        scoring="accuracy"
    ).mean()

    return score 


In [32]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Best Params:  ", study.best_params)
print("Best Score:  ", study.best_value)

[I 2026-02-12 11:39:55,604] A new study created in memory with name: no-name-ae00f66a-7693-4d68-99bf-935889cae3a1
[I 2026-02-12 11:39:55,666] Trial 0 finished with value: 0.8031993905922683 and parameters: {'C': 5.37535822317606, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.8031993905922683.
[I 2026-02-12 11:39:55,768] Trial 1 finished with value: 0.8031993905922683 and parameters: {'C': 9.33385627603089, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.8031993905922683.
[I 2026-02-12 11:39:55,795] Trial 2 finished with value: 0.8031993905922683 and parameters: {'C': 6.970447824811035, 'solver': 'liblinear'}. Best is trial 0 with value: 0.8031993905922683.
[I 2026-02-12 11:39:55,842] Trial 3 finished with value: 0.8031993905922683 and parameters: {'C': 8.964200485559749, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.8031993905922683.
[I 2026-02-12 11:39:55,889] Trial 4 finished with value: 0.8031993905922683 and parameters: {'C': 1.744145724020387, 'solver': 'lbfgs'}. Best is

Best Params:   {'C': 0.24838896716959802, 'solver': 'liblinear'}
Best Score:   0.8076937726147401


In [33]:
best_model = LogisticRegression(**study.best_params, max_iter=1000) 
best_model.fit(x_train, y_train)

best_pred = best_model.predict(x_test)
print("Tuning Accuracy Score: ", accuracy_score(y_test, best_pred))

Tuning Accuracy Score:  0.8089887640449438


# Modelni Saqlash 

In [35]:
import joblib 

joblib.dump(best_model, "Titanic_dataset.joblib")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

# Modelni yuklab olish 

In [36]:
loaded_model = joblib.load("Titanic_dataset.joblib")

# Modelni Papakga Yuklash

In [37]:
import joblib 
import os 

save_dir = r"C:\Users\Rasulbek907\Desktop\Project_One\Models"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, "Titanic_dataset.joblib")
scaler_path = os.path.join(save_dir, "scaler.joblib")

joblib.dump(best_model, model_path)
joblib.dump(scaler, scaler_path)

print(f"Model Saqlandi : {model_path}")
print(f"Scaler Saqlandi: {scaler_path}")

Model Saqlandi : C:\Users\Rasulbek907\Desktop\Project_One\Models\Titanic_dataset.joblib
Scaler Saqlandi: C:\Users\Rasulbek907\Desktop\Project_One\Models\scaler.joblib
